In [1]:
pip install torch torchvision matplotlib numpy torchinfo lightning


Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
from torchvision.transforms import ToTensor, Compose, Resize, Lambda, RandomHorizontalFlip, RandomRotation
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, random_split
import numpy as np
from matplotlib.pyplot import subplots
from torch import nn
from torchinfo import summary
import torch.nn.functional as F
import lightning as L
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint




In [7]:
import os
import pandas as pd

Cells_Data = pd.DataFrame(columns=["class", "filepath"])

for root, dirs, files in os.walk("archive"):
    if os.path.basename(root) == "CROPPED":
        class_name = os.path.basename(os.path.dirname(root))
        
        for file in files:
            if file.endswith(".bmp"):
                file_path = os.path.join(root, file)
                Cells_Data = pd.concat([Cells_Data, pd.DataFrame({
                    "class": [class_name],
                    "filepath": [file_path]
                })], ignore_index=True)

"Total de imagens .bmp:", len(Cells_Data)

('Total de imagens .bmp:', 4049)

In [9]:
random_split(Cells_Data)

TypeError: random_split() missing 1 required positional argument: 'lengths'

In [4]:
class BuildingBlock(nn.Module):
    def __init__(self ,
                    in_channels ,
                    out_channels):
        super(BuildingBlock , self).__init__ ()
        self.conv = nn.Conv2d(in_channels=in_channels ,
                                out_channels=out_channels ,
                                kernel_size =(3 ,3),
                                padding='same')
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.ReLU ()
        self.pool = nn.MaxPool2d(kernel_size =(2 ,2))
    def forward(self , x):
      return self.pool(self.activation(self.bn(self.conv(x))))

In [5]:
class CVD_Model(nn.Module):
    def __init__(self):
        super(CVD_Model , self).__init__ ()
        sizes = [(3 ,32) ,
        (32 ,64) ,
        (64 ,128) ,
        (128 ,256),
        (256, 512)]
        self.conv = nn.Sequential (*[ BuildingBlock(in_ , out_)
                                        for in_ , out_ in sizes ])
        self.output = nn.Sequential(nn.Dropout (0.5) ,
                                    nn.Linear (7*7*512 , 1024) ,
                                    nn.ReLU (),
                                    nn.Linear(1024, 512),
                                    nn.ReLU(),
                                    nn.Linear (512, 2))
    def forward(self , x):
        val = self.conv(x)
        val = torch.flatten(val , start_dim =1)
        return self.output(val)


In [6]:
class CVD_Small(nn.Module):
    """2 blocos conv + 2 lineares"""
    def __init__(self, num_classes=2):
        super().__init__()
        sizes = [(3, 32),   
                 (32, 64)]  
        self.conv = nn.Sequential(
            *[BuildingBlock(cin, cout) for cin, cout in sizes]
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # --- PARTE FULLY‑CONNECTED ---
        self.output = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(64, 128),  # 1ª camada linear
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)  # 2ª camada linear (saída)
        )
    def forward(self, x):
        x = self.conv(x)       # [B, 64, H/4, W/4]
        x = self.avgpool(x)        # [B, 64, 1, 1]
        x = torch.flatten(x, 1)    # [B, 64]
        x = self.output(x)     # [B, num_classes]
        return x



In [7]:
cvd_model = CVD_Model()
summary(cvd_model ,
input_data= torch.randn(1, 3, 224, 224),
col_names =['input_size',
'output_size',
'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
CVD_Model                                [1, 3, 224, 224]          [1, 2]                    --
├─Sequential: 1-1                        [1, 3, 224, 224]          [1, 512, 7, 7]            --
│    └─BuildingBlock: 2-1                [1, 3, 224, 224]          [1, 32, 112, 112]         --
│    │    └─Conv2d: 3-1                  [1, 3, 224, 224]          [1, 32, 224, 224]         896
│    │    └─BatchNorm2d: 3-2             [1, 32, 224, 224]         [1, 32, 224, 224]         64
│    │    └─ReLU: 3-3                    [1, 32, 224, 224]         [1, 32, 224, 224]         --
│    │    └─MaxPool2d: 3-4               [1, 32, 224, 224]         [1, 32, 112, 112]         --
│    └─BuildingBlock: 2-2                [1, 32, 112, 112]         [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-5                  [1, 32, 112, 112]         [1, 64, 112, 112]         18,496
│    │    └─BatchNorm2d: 3-6  

In [9]:
cvd_model_small = CVD_Small()
summary(cvd_model_small ,
input_data= torch.randn(1, 3, 224, 224),
col_names =['input_size',
'output_size',
'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
CVD_Small                                [1, 3, 224, 224]          [1, 2]                    --
├─Sequential: 1-1                        [1, 3, 224, 224]          [1, 64, 56, 56]           --
│    └─BuildingBlock: 2-1                [1, 3, 224, 224]          [1, 32, 112, 112]         --
│    │    └─Conv2d: 3-1                  [1, 3, 224, 224]          [1, 32, 224, 224]         896
│    │    └─BatchNorm2d: 3-2             [1, 32, 224, 224]         [1, 32, 224, 224]         64
│    │    └─ReLU: 3-3                    [1, 32, 224, 224]         [1, 32, 224, 224]         --
│    │    └─MaxPool2d: 3-4               [1, 32, 224, 224]         [1, 32, 112, 112]         --
│    └─BuildingBlock: 2-2                [1, 32, 112, 112]         [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-5                  [1, 32, 112, 112]         [1, 64, 112, 112]         18,496
│    │    └─BatchNorm2d: 3-6  

In [10]:
class LitCVDModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = CVD_Model() 
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        X, y = batch
        logits = self(X)
        loss = self.loss_fn(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        self.log('train_acc', acc, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y = batch
        logits = self(X)
        loss = self.loss_fn(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', acc, on_step=False, on_epoch=True)

    def test_step(self, batch, batch_idx):
        X, y = batch
        logits = self(X)
        loss = self.loss_fn(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('test_loss', loss, on_epoch=True)
        self.log('test_acc', acc, on_epoch=True)
        

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=0.001)


In [11]:
class LitCVDSmall(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = CVD_Small() 
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        X, y = batch
        logits = self(X)
        loss = self.loss_fn(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        self.log('train_acc', acc, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y = batch
        logits = self(X)
        loss = self.loss_fn(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', acc, on_step=False, on_epoch=True)

    def test_step(self, batch, batch_idx):
        X, y = batch
        logits = self(X)
        loss = self.loss_fn(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('test_loss', loss, on_epoch=True)
        self.log('test_acc', acc, on_epoch=True)
        

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=0.001)

In [ ]:
model = LitCVDModel()


checkpoint_callback = ModelCheckpoint(
    monitor="val_acc",
    mode="max",
    save_top_k=1,
    filename="best-cvd-model",
    verbose=True
)

trainer = Trainer(
    accelerator="auto",  
    devices=1,
    max_epochs=120,
    precision=16,         
    log_every_n_steps=10,
    callbacks=[checkpoint_callback]
    
)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
